# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.

__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

In [3]:
# Added cell to set the working directory to be Assignments/HW1
%cd Assignments/HW1
%pwd

/media/notebooks/sp21-andreaslemos/Assignments/HW1


'/media/notebooks/sp21-andreaslemos/Assignments/HW1'

Create a folder for any data you download locally.

In [4]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

> My name is Andreas Lemos. I just moved - 3 months ago - from Brazil to Orlando, FL. I am half way through the MIDS course and this is my first advanced course. This course has direct applicability to my work as we are trying to setup a data collection pipeline for BigData. I am excited to learn the material and aplly it to real-world problems at work. In addition, I have recently run into issues with some of my work models taking a long time to run. If there is a way to apply parallel processing to them, it would also be a big help.

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> There are two main types of errors inherent in any model. The first one is the Reducible error, and the second is the Irreducible Error. Looking at the formula for the Error, we have that:

$$Err_{(x)} = \Big(E[\hat{f}(x)]-f(x)\Big)^2 + E\Big[\big(\hat{f}(x)-E(\hat{f}(x)]\big)^2\Big]+\sigma_e^2$$

> This can be interpreted as:

$$Err(x) = Bias^2 + Variance + Irreducible Error$$

> The irreductible Error is the noise term and cannot be reduced by the model. The Reducible Error then has two parts to it, the Square of the Bias and the Variance. In order to minimize the total error, we must then try to minimize the quantity $Bias^2 + Variance$

> In real world applications with imperfect models, the tradeoff between Bias and Variance means that when you try to reduce the Model Error from Bias, for example, the Error from Variance will increase, and vice-versa. What you have to do then is tune the model so you find the minimal total error (sum of the Square of Bias and Variance).

> In general, as the model complexity increases, the Error due to Bias reduces, and the Error due to Variance increases.

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ If you use a tokenizer that is case sensitive, meaning that it does not convert everything to lowercase, for example, the calucaltion would be different, as shown below:

> Tokenizer - case sensitive: 

>> total unique words = 9 (since 'Alice' is not equal to 'alice')

>> likelihood of word 'alice' = 1/9

> Tokenizer - all lowercase: 

>> total unique words = 8 (since 'Alice would equal 'alice')

>> likelihood of word 'alice' = 2/8


> __b)__ The function removes punctuation, discards numerical digits, makes everything lowercase and only considers consecutive a-z letters. Therefore, tokenizing ("By-the-bye, what became of Alice's 12 hats?!") would produce: 

>> 'by' 'the' 'bye' 'what' 'became' 'of' 'alice' 's' 'hats'

>> Please note that the quotes above are just to separate the output strings and for clarity only. Also, it is assumed that the objective was to remove 'hiphens', for example, and consider each word individually. So hiphens are 'replaced' by spaces.

In [5]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[a-z]+")          

In [6]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [7]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   234k      0 --:--:-- --:--:-- --:--:--  234k


In [8]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

﻿The Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [9]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Writing data/alice_test.txt


In [10]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._

> __d)__ I think the answer depends on the situation. If you are doing a one time analysis, you may prefer to post-process the word and its plurals, since there may be a litmited set of them. However, if this is an application that will be run several times with different inputs, then it may be better to start the analysis over with a new tokenizer that takes plurals into account. That is beacuse then all the processing of the data happens at the same 'step' of the process.

In [11]:
# part a - DO YOUR WORK IN wordCount.py

In [12]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this 	 3
is 	 2
a 	 2
small 	 3
test 	 3
file 	 3
for 	 1
has 	 1
two 	 1
lines 	 1


In [13]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [14]:
!head data/alice_counts.txt

the 	 1837
project 	 88
gutenberg 	 98
ebook 	 12
of 	 637
alice 	 404
s 	 224
adventures 	 12
in 	 437
wonderland 	 8


__Check your results:__ How many times does the word "alice" appear in the book? 

In [15]:
# EXPECTED OUTPUT: 404
!grep alice data/alice_counts.txt

alice 	 404


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [16]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter 	 56
hatters 	 1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [17]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen 	 76
queens 	 1


# Question 5: Unix Sorting Practice
Another common task in this course's assignments will be to make strategic use of sorting.     

### Q5 Tasks:
* __a) short response:__ What is the Big O complexity of the fastest sorting algorithms? When we take the time to sort into account, does it make sense to alphabetize before combining singular-plural pairs? [*HINT: If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).*]

* __b) short response:__ What is the default sorting algorithm in MapReduce? What is the Big O complexity of this algorithm? Why do you think this algorithm was chosen? [*HINT: Julius Ceasar! (week 1 slides)*]

* __c) code:__ write a unix command to check how many records are in your word count file.

* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [*HINT: if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)*]

* __e) code:__ Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ The fastest sorting algorithm, at best, can have a complexity of $O(n)$. However, that is not possible in the average case, where the best algorithms are $O(n+k)$

>> Yes, it makes sense to alphabetize before combining singular-plural pairs. The reason being that two algorithms in sequence, one being $O(n \cdot log(n))$ and the other $O(n)$ will perfom faster than one $O(n^2)$ algorithm.


> __b)__ The default sorting algorithm for MapReduce is Merge Sort. It's overall time complexity is $O(n \cdot log(n))$, with a space complexity of $O(n)$. I thik this is because of its efficiency and also the fact that it is a stable sort, meaning the same element in an array maintains its original position with respect to each other. 


In [18]:
# part c - write a unix command to check how many records are in your word count file
!wc -l data/alice_counts.txt

3021 data/alice_counts.txt


In [19]:
# part d - unix command to sort your word counts alphabetically 
!sort -k1,1 data/alice_counts.txt > data/alice_counts_A-Z.txt

In [20]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a 	 695
abide 	 2
able 	 1
about 	 102
above 	 3
absence 	 1
absurd 	 2
accept 	 1
acceptance 	 1
accepted 	 2


In [21]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -n -r -k2,2 data/alice_counts.txt > data/alice_counts_sorted.txt

In [22]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the 	 1837
and 	 946
to 	 811
a 	 695
of 	 637
it 	 610
she 	 553
i 	 546
you 	 487
said 	 462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1837
and	946
to	811
a	695
of	637
it	610
she	553
i	546
you	487
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ The command listed below for part '4b' did not include the optional 'reducer' file parameter ("reducer.py = an executable script after the parallel processes are complete.").  Therefore, all parallel counts were included in a single file, but duplicate 'word' lines were not combined into a single line with the total count.

In [23]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [24]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [25]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice 	 113
alice 	 126
alice 	 122
alice 	 43


In [26]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [27]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [28]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	404


In [29]:
!man sort

No manual entry for sort


# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ In aggregateCounts_v1.py, each line needs to be compared against all others in the file. This has a complexity of $O(n^2)$, which does not scale well. For example, if you only have 100 lines in the file, that means a total number of 5,050 comparisons (see below). However, this number quickly increases to over 12 millions comparisons if the line count is 5,000.
$$Comparisions Needed = \frac{n*(n+1)}{2}=\frac{(100*101)}/2=5,050$$

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ For the case of splitting the words counts after sorting them, and then perfrom the reducing in two separate machines, one of the potential issues that comes to mind is just how (and where) the file gets split. If the split is done in such a fashion that the same word exists in each split file, then there will be a miscount of that word when the reuslts are put back togehter. 

> __e)__ In order to avoid this sort of problem, one could take an approach similar to 'shuffling' in MapReduce. What you'd do in this case is to assign a 'super key' for a group of words. One could, for example, assign a key of "G1" to all words that start with the letters a through l. And "G2" to all letter that start with the letters m through z. That way you can ensure that all 'duplicated' words would end up in the same machine for the 'reduce' step.

In [4]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
# !chmod a+x aggragateCounts_v2.sh # original line from HW
!chmod a+x aggregateCounts_v2.py # modified line by student since this is a .py file, nort a .sh one, and a spelling error

In [5]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [6]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [7]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	404


# Question 8: Scalability Considerations
In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ The two features of an 'ideal algorithm' are the scalability in terms of data and in terms of resources. For a data perspective, an algorithm should be able to run twice as much data in no more than twice the amount of time. Similarly, given twice the resources (i.e., cluster size), an ideal algorithm should runs at least twice as fast. Furthermore, these scalability features should hold true over a wide range of settings and with no need to change any of the algorithm's parameters.

> __b)__ In the case for this simple algorithm, and for the bounds given here (1, 2, and 4 partitions), the algorithm does get close to meeting Lin and Dyer's criteria. But it falls short. Under Lin and Dyer's criteria, one would expect that increasing the Partitions from 1 to 4 would mean the total time would be reduced by at least 1/4. However, in this case it reduced from 4.46s to 1.23s, not to 1.115s. 

| Partitions | Time (seconds) |
|------------|----------------|
| 1          | 4.46           |
| 2          | 2.30           |
| 4          | 1.23           |


> __c)__ When looking at the algorithm for our Word Count analysis, the criteria for an ideal algorithm does not hold. There are several elements at play here. First, the time unit is much about an order of magnitude smaller than the example above. This means that any communication costs increase with the higher number of partitions and it has a impact on the total time, given that they are in microseconds. This communication cost was probably not as evident on the previous example as the algorithm was not optimized, making the processing time of each individual process be the dominant factor. In the example, that does not happen, and the communication costs does have a visible impact on the overall times. We see that 2 or 4 partitions is about the ideal number. More partitions than that and the communications costs start to take over and dominate the total time. 

| Partitions | Time (miliseconds) |
|------------|--------------------|
| 1          | 212                |
| 2          | 199                |
| 4          | 200                |
| 8          | 233                |
| 16         | 296                |
| 32         | 433                |

> __d)__ The tradeoff here seems to be parallelism and communication costs. While there is a reduction in the processing time for each partition individually as the number of partitions increase, the costs associated with communication seems to be the prevalent facotr as the number of partiions go to 8 or higher.  

__Run the following cells to create the mapper referenced in `part b`__

In [94]:
!mkdir demo

In [95]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Writing demo/mapper.py


In [96]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [97]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

4.46 s ± 3.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.3 s ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.23 s ± 5.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [100]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

212 ms ± 798 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

199 ms ± 430 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

200 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [103]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

233 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

296 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [105]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

433 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Chapter 2 in DITP_ ]

### Q9 Student Answers:

> __a)__ An Embarrissingly Parallel problem is one that can be separated out into multiple parallel tasks with little to no effort. In the case of word count, a text can be easily divided into smaller 'pieces' and each piece can be processed independently of one another. That's an example of an Embarrassingly Parallel problem. The term refers to a 'task', meaning that the tasks of counting words, for example, lends itself to have independent, smaller chunks to be processed. 

> __b)__ For any functions that are associative commutative, the problem becomes and Embarrassingly Parallel one. That is becuase of the properties associated with these two terms. A commutative problem is one where $(a+b)=(b+a)$. An associated problem is one where $(a+b)+c=a+(b+c)$. This two properties guarantee an Embarrassingly Parallel operation since the problem can be broken down and put together in different order with no impact to the ouput. 

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform